<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [8]:
#Factorizacion QR
def QR(A):
    n=len(A);
    Q=np.zeros((n,n))
    R=np.zeros((n,n))
    R[0,0]=np.linalg.norm(A[:,0])
    Q[:,0]=A[:,0]/np.linalg.norm(A[:,0])
    # A_k=r_{1l}*q_1+\cdots+ r_{kk}*q_{k} entonces r_{ik}=q_{i} \cdot a_{k}
    #q_{k}=(a_k-\sum^{k-1} r_{ik}*q_{i})/norma.
    for k in range(1,n):
        a_star=np.array(A[:,k])
        for i in range(k):
            R[i,k]=np.dot(Q[:,i],A[:,k])
            a_star=np.subtract(a_star,R[i,k]*Q[:,i])
        Q[:,k]=a_star/np.linalg.norm(a_star)
        R[k,k]=np.linalg.norm(a_star)
    return(Q,R)
def check_diag(A,tol):#Pensado para matrices en general (NO simetricas)
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)

In [ ]:
A=np.array([[1,1,1],[0,5,2],[-1,2,1]])
Q,R=QR(A)
print(np.dot(Q,R))

[[ 1.  1.  1.]
 [ 0.  5.  2.]
 [-1.  2.  1.]]


In [38]:
A=np.loadtxt("Eigen_3.txt",skiprows=1)
Q,R=QR(A)

In [48]:
def QR_iteration(A,tol=10**(-6),max_iter=10):
    Q,R=QR(A)
    A=np.dot(R,Q)
    count=0
    phi=Q
    while(check_diag(A,tol)==False & (count<max_iter)):
        count+=1
        Q,R=QR(A)
        phi=np.dot(phi,Q)
        A=np.dot(R,Q)
    if(count==max_iter):
        print("max iter")
    else:
        return [phi,np.diag(A)]

In [55]:
Q,D=QR_iteration(A)

In [57]:
print("Verificacion : ", np.dot(np.dot(Q.T,A),Q))

Verificacion :  [[ 3.61363835e+00  5.44282437e-15 -3.21371742e-15]
 [ 3.19545569e-16  7.19555862e-01 -7.09161734e-07]
 [-3.99596841e-18 -7.09161733e-07  4.74661293e-01]]


In [18]:
np.linalg.eig(A)

(array([3.61363835, 0.71955586, 0.47466129]),
 array([[ 0.1864669 , -0.95093899,  0.24687067],
        [-0.96209818, -0.22763778, -0.15016034],
        [-0.19899041,  0.20951389,  0.95734359]]))